# Is Deep Learning *slipping* [on the shoulders of Giants](https://www.wikiwand.com/en/Standing_on_the_shoulders_of_giants)?
### A critical reproducibility challenge for [[Rumelhart et al., 1986]](https://sci-hub.se/10.1038/323533a0)

*TO BE FILLED*

Explain why and when it all started...

In [ ]:
# ===========
# IMPORTS
# ===========

# Type hints
from typing import Iterable, List, Tuple
from torch import Tensor

# Tensors and NNs
import torch as th

# Tensor data[sets|loader]s
from torch.utils.data import TensorDataset, DataLoader

# Iterable handling
from itertools import product



In [ ]:
# The algorithmic function (symmetry-detection for 5-sized binary inputs) we
# want to approximate with a NN
def is_symmetric(iterable: Iterable) -> float:
    assert len(iterable) == 6
    if (
        # iterable[0:3] == iterable[5:2:-1] still unsupported for PyTorch tensors
        iterable[0] == iterable[-1]
        and iterable[1] == iterable[-2]
        and iterable[2] == iterable[-3]
    ):
    # 1 == Yes | 0 == No
        return 1.0
    return 0.0

In [ ]:
# We split the dataset output-wise early on, to be able to balance it later in
# case we need to.

x_all: List[Tuple[float]] = [item for item in product([0.0, 1.0], repeat=6)]
x_symmetric: List[Tuple[float]] = [item for item in x_all if is_symmetric(item)]
x_non_symmetric: List[Tuple[float]] = [item for item in set(x_all).difference(set(x_symmetric))]

# And we tensorize it
del x_all   # We don't need it anymore
x_symmetric: Tensor = th.tensor(x_symmetric, dtype=th.float32)
x_non_symmetric: Tensor = th.tensor(x_non_symmetric, dtype=th.float32)

In [ ]:
# The original dataset tensor
x: Tensor = th.cat((x_non_symmetric, x_symmetric), dim=0)
y: Tensor = th.tensor([[is_symmetric(sub_x)] for sub_x in x])

# And the balanced one
balancing_ratio: int = int((x_non_symmetric.shape[0]/x_symmetric.shape[0]))
x_balanced: Tensor = th.cat((x_non_symmetric, th.cat([x_symmetric]*balancing_ratio, dim=0)), dim=0)
y_balanced: Tensor = th.tensor([[is_symmetric(sub_x)] for sub_x in x_balanced])
del balancing_ratio

In [ ]:
# Conversion to proper PyTorch data[set|loader]s

# Datasets
train_unbalanced_ds = TensorDataset(x, y)
train_balanced_ds = TensorDataset(x_balanced, y_balanced)

# Dataloaders (we do full-dataset-batching as in the paper)
train_unbalanced_dl = DataLoader(train_unbalanced_ds, batch_size=len(train_unbalanced_ds), shuffle=True)
train_balanced_dl = DataLoader(train_balanced_ds, batch_size=len(train_balanced_ds), shuffle=True)